In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_from_disk

# Load the fine-tuned model
model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/LHL/llm-project/tuned_distilbert_model"
)
tokenizer = AutoTokenizer.from_pretrained(
    "/content/drive/MyDrive/LHL/llm-project/tuned_distilbert_model"
)

# Load the dataset
tokenized_ds = load_from_disk("/content/drive/MyDrive/LHL/llm-project/tuned_tokenized_ds")

In [4]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
import numpy as np

training_args = TrainingArguments(
    output_dir="./distilbert_optimized",  # New output directory
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,  # ✅ Keeping 2 epochs
    learning_rate=3e-5,  # ✅ Lower learning rate (from 5e-5 to 3e-5)
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
# Define compute_metrics function again
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Convert logits to label predictions
    return {"accuracy": accuracy_score(labels, predictions)}

# Reinitialize Trainer with updated learning rate
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["train"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start fine-tuning
trainer.train()

<ipython-input-5-bd3afea0e36b>:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.234800,0.150950,0.945610
2,0.155000,0.089585,0.970940


TrainOutput(global_step=2414, training_loss=0.19489876179035379, metrics={'train_runtime': 547.0485, 'train_samples_per_second': 70.579, 'train_steps_per_second': 4.413, 'total_flos': 5114566262108160.0, 'train_loss': 0.19489876179035379, 'epoch': 2.0})

In [6]:
# Run model predictions on the test set
test_predictions = trainer.predict(tokenized_ds["test"])

import numpy as np
from sklearn.metrics import accuracy_score

# Convert logits to label predictions (0 = Negative, 1 = Positive)
predicted_labels = np.argmax(test_predictions.predictions, axis=-1)

# Extract actual labels from test dataset
true_labels = tokenized_ds["test"]["label"]

# Compute accuracy
test_accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.7171


In [7]:
from transformers import AutoModelForSequenceClassification

# Reload DistilBERT from the last saved checkpoint (ensuring a fresh start)
model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/LHL/llm-project/tuned_distilbert_model"
)

In [8]:
training_args = TrainingArguments(
    output_dir="./distilbert_optimized_7e5",  # New output directory
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,  # ✅ Keeping 2 epochs
    learning_rate=7e-5,  # ✅ Increased learning rate (from 3e-5 to 7e-5)
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["train"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start fine-tuning with updated learning rate
trainer.train()

<ipython-input-9-dfad772145a6>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.323200,0.191245,0.925408
2,0.193800,0.075788,0.973271


TrainOutput(global_step=2414, training_loss=0.25850264731772993, metrics={'train_runtime': 546.9899, 'train_samples_per_second': 70.586, 'train_steps_per_second': 4.413, 'total_flos': 5114566262108160.0, 'train_loss': 0.25850264731772993, 'epoch': 2.0})

In [10]:
# Run model predictions on the test set
test_predictions = trainer.predict(tokenized_ds["test"])

import numpy as np
from sklearn.metrics import accuracy_score

# Convert logits to label predictions (0 = Negative, 1 = Positive)
predicted_labels = np.argmax(test_predictions.predictions, axis=-1)

# Extract actual labels from test dataset
true_labels = tokenized_ds["test"]["label"]

# Compute accuracy
test_accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.7075


In [11]:
from transformers import AutoModelForSequenceClassification

# Reload DistilBERT from the last saved checkpoint (ensuring a fresh start)
model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/LHL/llm-project/tuned_distilbert_model"
)

In [12]:
training_args = TrainingArguments(
    output_dir="./distilbert_optimized_batch",  # New output directory
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=8,  # ✅ Adjusted batch size (from 16 → 8 or 32)
    per_device_eval_batch_size=8,  # ✅ Matching eval batch size
    num_train_epochs=2,  # Keeping at 2 epochs
    learning_rate=5e-5,  # Resetting to best LR so far
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["train"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start fine-tuning with updated learning rate
trainer.train()

<ipython-input-13-dfad772145a6>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.438600,0.243862,0.908780
2,0.277700,0.070358,0.978814


TrainOutput(global_step=4828, training_loss=0.35812677543027005, metrics={'train_runtime': 593.7824, 'train_samples_per_second': 65.024, 'train_steps_per_second': 8.131, 'total_flos': 5114566262108160.0, 'train_loss': 0.35812677543027005, 'epoch': 2.0})

In [14]:
# Run model predictions on the test set
test_predictions = trainer.predict(tokenized_ds["test"])

import numpy as np
from sklearn.metrics import accuracy_score

# Convert logits to label predictions (0 = Negative, 1 = Positive)
predicted_labels = np.argmax(test_predictions.predictions, axis=-1)

# Extract actual labels from test dataset
true_labels = tokenized_ds["test"]["label"]

# Compute accuracy
test_accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.7203


In [15]:
from transformers import AutoModelForSequenceClassification

# Reload DistilBERT from the last saved checkpoint (ensuring a fresh start)
model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/LHL/llm-project/tuned_distilbert_model"
)

In [16]:
training_args = TrainingArguments(
    output_dir="./distilbert_optimized_batch",  # New output directory
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=32,  # ✅ Adjusted batch size (from 16 → 8 or 32)
    per_device_eval_batch_size=32,  # ✅ Matching eval batch size
    num_train_epochs=2,  # Keeping at 2 epochs
    learning_rate=5e-5,  # Resetting to best LR so far
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["train"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start fine-tuning with updated learning rate
trainer.train()

<ipython-input-17-dfad772145a6>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.351300,0.157809,0.952914
2,0.163100,0.054178,0.984201


TrainOutput(global_step=1208, training_loss=0.25724611219191396, metrics={'train_runtime': 517.1725, 'train_samples_per_second': 74.656, 'train_steps_per_second': 2.336, 'total_flos': 5114566262108160.0, 'train_loss': 0.25724611219191396, 'epoch': 2.0})

In [18]:
# Run model predictions on the test set
test_predictions = trainer.predict(tokenized_ds["test"])

import numpy as np
from sklearn.metrics import accuracy_score

# Convert logits to label predictions (0 = Negative, 1 = Positive)
predicted_labels = np.argmax(test_predictions.predictions, axis=-1)

# Extract actual labels from test dataset
true_labels = tokenized_ds["test"]["label"]

# Compute accuracy
test_accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.7184


In [19]:
from transformers import AutoModelForSequenceClassification

# Reload DistilBERT from the last saved checkpoint (ensuring a fresh start)
model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/LHL/llm-project/tuned_distilbert_model"
)

In [20]:
training_args = TrainingArguments(
    output_dir="./distilbert_optimized_batch",  # New output directory
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=32,  # ✅ Adjusted batch size (from 16 → 8 or 32)
    per_device_eval_batch_size=32,  # ✅ Matching eval batch size
    num_train_epochs=2,  # Keeping at 2 epochs
    learning_rate=5e-5,  # Resetting to best LR so far
    weight_decay=0.02,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["train"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start fine-tuning with updated learning rate
trainer.train()

<ipython-input-21-dfad772145a6>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.351200,0.156546,0.953846
2,0.163000,0.054143,0.984201


TrainOutput(global_step=1208, training_loss=0.25707916866075126, metrics={'train_runtime': 518.1046, 'train_samples_per_second': 74.522, 'train_steps_per_second': 2.332, 'total_flos': 5114566262108160.0, 'train_loss': 0.25707916866075126, 'epoch': 2.0})

In [22]:
# Run model predictions on the test set
test_predictions = trainer.predict(tokenized_ds["test"])

import numpy as np
from sklearn.metrics import accuracy_score

# Convert logits to label predictions (0 = Negative, 1 = Positive)
predicted_labels = np.argmax(test_predictions.predictions, axis=-1)

# Extract actual labels from test dataset
true_labels = tokenized_ds["test"]["label"]

# Compute accuracy
test_accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.7197


In [23]:
from transformers import AutoModelForSequenceClassification

# Reload DistilBERT from the last saved checkpoint (ensuring a fresh start)
model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/LHL/llm-project/tuned_distilbert_model"
)

In [24]:
training_args = TrainingArguments(
    output_dir="./distilbert_optimized_warmup",  # New output directory
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    learning_rate=5e-5,
    weight_decay=0.01,  # Resetting weight decay to default since 0.02 didn't help
    warmup_steps=500,  # ✅ New adjustment: Gradual learning rate warmup
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["train"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-25-7135b41bcad9>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.319200,0.212023,0.915721
2,0.223700,0.075382,0.979539


TrainOutput(global_step=1208, training_loss=0.2714534026897506, metrics={'train_runtime': 517.476, 'train_samples_per_second': 74.612, 'train_steps_per_second': 2.334, 'total_flos': 5114566262108160.0, 'train_loss': 0.2714534026897506, 'epoch': 2.0})

In [26]:
# Run model predictions on the test set
test_predictions = trainer.predict(tokenized_ds["test"])

import numpy as np
from sklearn.metrics import accuracy_score

# Convert logits to label predictions (0 = Negative, 1 = Positive)
predicted_labels = np.argmax(test_predictions.predictions, axis=-1)

# Extract actual labels from test dataset
true_labels = tokenized_ds["test"]["label"]

# Compute accuracy
test_accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.7286


In [28]:
from transformers import AutoModelForSequenceClassification

# Reload DistilBERT from the last saved checkpoint (ensuring a fresh start)
model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/LHL/llm-project/tuned_distilbert_model"
)

In [30]:
training_args = TrainingArguments(
    output_dir="./distilbert_optimized_dropout",  # New output directory
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:
from transformers import AutoModelForSequenceClassification

# Reload DistilBERT with increased dropout
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english",
    num_labels=2
)

# Manually adjust dropout layers
model.config.hidden_dropout_prob = 0.3  # ✅ Increase from 0.2 → 0.3
model.config.attention_probs_dropout_prob = 0.3  # ✅ Increase from 0.2 → 0.3

In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["train"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-32-7135b41bcad9>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.638700,0.447644,0.803989
2,0.435100,0.255859,0.909298


TrainOutput(global_step=2414, training_loss=0.5369317561215177, metrics={'train_runtime': 545.0304, 'train_samples_per_second': 70.84, 'train_steps_per_second': 4.429, 'total_flos': 5114566262108160.0, 'train_loss': 0.5369317561215177, 'epoch': 2.0})

In [33]:
# Run model predictions on the test set
test_predictions = trainer.predict(tokenized_ds["test"])

import numpy as np
from sklearn.metrics import accuracy_score

# Convert logits to label predictions (0 = Negative, 1 = Positive)
predicted_labels = np.argmax(test_predictions.predictions, axis=-1)

# Extract actual labels from test dataset
true_labels = tokenized_ds["test"]["label"]

# Compute accuracy
test_accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.7305


In [34]:
from transformers import AutoModelForSequenceClassification

# Reload DistilBERT with best-performing dropout setting (0.2)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english",
    num_labels=2
)

# Set dropout back to 0.2
model.config.hidden_dropout_prob = 0.2
model.config.attention_probs_dropout_prob = 0.2

In [35]:
training_args = TrainingArguments(
    output_dir="./distilbert_optimized_dropout",  # New output directory
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [36]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["train"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-36-7135b41bcad9>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.638100,0.455428,0.803315
2,0.437100,0.258410,0.907692


TrainOutput(global_step=2414, training_loss=0.5376040277741818, metrics={'train_runtime': 546.79, 'train_samples_per_second': 70.612, 'train_steps_per_second': 4.415, 'total_flos': 5114566262108160.0, 'train_loss': 0.5376040277741818, 'epoch': 2.0})

In [37]:
# Run model predictions on the test set
test_predictions = trainer.predict(tokenized_ds["test"])

import numpy as np
from sklearn.metrics import accuracy_score

# Convert logits to label predictions (0 = Negative, 1 = Positive)
predicted_labels = np.argmax(test_predictions.predictions, axis=-1)

# Extract actual labels from test dataset
true_labels = tokenized_ds["test"]["label"]

# Compute accuracy
test_accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.7318


In [38]:
model.save_pretrained("/content/drive/MyDrive/LHL/llm-project/best_distilbert_model")
tokenizer.save_pretrained("/content/drive/MyDrive/LHL/llm-project/best_distilbert_model")

('/content/drive/MyDrive/LHL/llm-project/best_distilbert_model/tokenizer_config.json',
 '/content/drive/MyDrive/LHL/llm-project/best_distilbert_model/special_tokens_map.json',
 '/content/drive/MyDrive/LHL/llm-project/best_distilbert_model/vocab.txt',
 '/content/drive/MyDrive/LHL/llm-project/best_distilbert_model/added_tokens.json',
 '/content/drive/MyDrive/LHL/llm-project/best_distilbert_model/tokenizer.json')

In [39]:
from huggingface_hub import notebook_login

notebook_login()

In [40]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [41]:
from transformers import Trainer, TrainingArguments

repo_name = "best_distilbert_model"  # Change this if you want a different name

# Define training arguments for Hub upload
training_args = TrainingArguments(
   output_dir=repo_name,
   push_to_hub=True
)

# Push model to Hugging Face Hub
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer
)

trainer.push_to_hub()

<ipython-input-41-64295cfaf013>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/colterwood/best_distilbert_model/commit/9aaf5cca7578719bd607ed4b2c89bae0c911eb0e', commit_message='End of training', commit_description='', oid='9aaf5cca7578719bd607ed4b2c89bae0c911eb0e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/colterwood/best_distilbert_model', endpoint='https://huggingface.co', repo_type='model', repo_id='colterwood/best_distilbert_model'), pr_revision=None, pr_num=None)

In [42]:
from transformers import pipeline

# Load the model from the Hub
my_model = pipeline("sentiment-analysis", model="colterwood/best_distilbert_model")

# Example text to test
data = [
    "This album was absolutely incredible. I loved every track!",
    "The production quality was disappointing, and the lyrics felt uninspired.",
]

# Get predictions
predictions = my_model(data)

# Show results
for text, pred in zip(data, predictions):
    print(f"Review: {text}\nPrediction: {pred}\n")

config.json:   0%|          | 0.00/881 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cuda:0


Review: This album was absolutely incredible. I loved every track!
Prediction: {'label': 'POSITIVE', 'score': 0.9991810917854309}

Review: The production quality was disappointing, and the lyrics felt uninspired.
Prediction: {'label': 'NEGATIVE', 'score': 0.9979774355888367}

